In [1]:
import pandas as pd

In [2]:
reviews  = pd.read_excel("customer_reviews.xlsx")

In [ ]:
reviews.ReviewText.sample(10)

In [ ]:
reviews['ReviewText'] = reviews['ReviewText'].str.replace("  ", " ")
reviews['ReviewText']

In [ ]:
# !pip install nltk

In [7]:
import nltk

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
# Download the VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')
# The VADER lexicon is a list of words and phrases with their corresponding sentiment 
# ratings that's used in the VADER (Valence Aware Dictionary and sEntiment Reasoner) sentiment analysis tool.

In [15]:
sia = SentimentIntensityAnalyzer()

### A polarity score is a numerical rating that measures the overall tone of a text, phrase, or word, and indicates whether it is positive, negative, or neutral. Polarity scores are used in sentiment analysis, which is the process of analyzing opinions
### Returns the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)

In [18]:
# Define a function to calculate sentiment scores using VADER
def calculate_sentiment(review):
    # Get the sentiment scores for the review text
    sentiment = sia.polarity_scores(review)
    # Return the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)
    # print(sentiment)
    return sentiment['compound']
    # return sentiment

In [25]:
# test code
review = "I really really really like this product.This product is such goose bump!"

# Calculate sentiment
print(calculate_sentiment(review))  # Outputs a compound score, e.g., 0.84

0.5613


In [ ]:
# Example test code
texts = [
    "Such a great invention. I am in love this product!",     # Positive
    "This product is okay.",    # Neutral
    "I hate this product."      # Negative
]

# Analyze and print the compound scores
for text in texts:
    scores = sia.polarity_scores(text)
    print(f"Text: {text}\nCompound Score: {scores['compound']}\n")


In [ ]:
# Apply sentiment analysis to calculate sentiment scores for each review
reviews['SentimentScore'] = reviews['ReviewText'].apply(calculate_sentiment)
reviews

In [28]:
# Define a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    # Use both the text sentiment score and the numerical rating to determine sentiment category
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment


In [ ]:
# Apply sentiment categorization using both text and rating
reviews['SentimentCategory'] = reviews.apply(lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)
reviews

In [30]:
# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

In [ ]:
# Apply sentiment bucketing to categorize scores into defined ranges
reviews['SentimentBucket'] = reviews['SentimentScore'].apply(sentiment_bucket)
reviews

In [32]:
reviews.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)

In [33]:
reviews.SentimentCategory.value_counts()

SentimentCategory
Positive          840
Negative          226
Mixed Negative    196
Mixed Positive     86
Neutral            15
Name: count, dtype: int64

In [54]:
twitter = pd.read_csv(r"G:\Saylani2024\Batch8\DataAnalysis\twitter_training.csv\twitter_training.csv",header=None)

In [55]:
twitter = twitter.iloc[:,3]
twitter

0        im getting on borderlands and i will murder yo...
1        I am coming to the borders and I will kill you...
2        im getting on borderlands and i will kill you ...
3        im coming on borderlands and i will murder you...
4        im getting on borderlands 2 and i will murder ...
                               ...                        
74677    Just realized that the Windows partition of my...
74678    Just realized that my Mac window partition is ...
74679    Just realized the windows partition of my Mac ...
74680    Just realized between the windows partition of...
74681    Just like the windows partition of my Mac is l...
Name: 3, Length: 74682, dtype: object

In [56]:
twitter = pd.Series(twitter,name="Review")

In [58]:
twitter = twitter.to_frame()

In [59]:
twitter

,Review
0,im getting on borderlands and i will murder yo...
1,I am coming to the borders and I will kill you...
2,im getting on borderlands and i will kill you ...
3,im coming on borderlands and i will murder you...
4,im getting on borderlands 2 and i will murder ...
...,...
74677,Just realized that the Windows partition of my...
74678,Just realized that my Mac window partition is ...
74679,Just realized the windows partition of my Mac ...
74680,Just realized between the windows partition of...


In [60]:
twitter['SentimentScore'] = twitter['Review'].apply(calculate_sentiment)
twitter

AttributeError: 'float' object has no attribute 'encode'